In this project we're trying to make a spam detection model for emails using LSTM


In [8]:
#First we Import the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D,LSTM,Embedding,TextVectorization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [9]:
#Formatting the dataset
df = pd.read_csv("spam.csv",encoding = "ISO-8859-1") #we load our dataset
#We remove the existing columns,add label and data columns
df = df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1)
df.columns = ["label","data"]
df["b_label"] = df["label"].map({"ham": 0,"spam":1}) #we add a binary columns and map it, if ham -> 0, if spam -> 1
df.head()

,label,data,b_label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
Y = df["b_label"].values
df_train,df_test,Ytrain,Ytest = train_test_split(df["data"],Y,train_size = 0.33) #we split our dataset into test and training

In [11]:
#create df dataset
train_ds = tf.data.Dataset.from_tensor_slices((df_train.values,Ytrain))
test_ds = tf.data.Dataset.from_tensor_slices((df_test.values,Ytest))
train_ds.map(lambda x,y:x)

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [12]:
#convert sentence to sequence
MAX_VOCAB_SIZE = 20_000
vectorization = TextVectorization(max_tokens=MAX_VOCAB_SIZE)
vectorization.adapt(train_ds.map(lambda x,y:x))

In [13]:
#shuffle and batch dataset
train_ds = train_ds.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [14]:
V = len(vectorization.get_vocabulary())

In [15]:
#Create the Model

#We get to choose Embedding dimensionality
D = 20

#Hidden state dimensionality
M = 15

#if string not specified it will return float casting error
i = Input(shape = (),dtype=tf.string)
x = vectorization(i)
x = Embedding(V,D)(x)
x = LSTM(M,return_sequences = True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1,activation = "sigmoid")(x)

model = Model(i,x)


In [16]:
#Compiling the model
model.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [17]:
model.fit(train_ds,validation_data = test_ds,epochs = 10)

Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.8445 - loss: 0.5309 - val_accuracy: 0.8653 - val_loss: 0.3883
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.8730 - loss: 0.3746 - val_accuracy: 0.8653 - val_loss: 0.3675
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.8842 - loss: 0.3171 - val_accuracy: 0.9671 - val_loss: 0.1913
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9816 - loss: 0.1407 - val_accuracy: 0.9681 - val_loss: 0.1320
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9918 - loss: 0.0819 - val_accuracy: 0.9652 - val_loss: 0.1225
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9972 - loss: 0.0473 - val_accuracy: 0.9486 - val_loss: 0.1526
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9955 - loss: 0.0428 - val_accuracy: 0.9687 - val_loss: 0.1271
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9953 - loss: 0.0412 - val_accuracy: 0.9700 - v

In [18]:

def predict_spam(text):

    text_tensor = tf.constant([text])
    prob = model.predict(text_tensor)[0][0]
    if prob > 0.5:
        return f"The email is a spam ({prob:.2f} confidence)"
    else:
        return f"The email in not spam ({1-prob:.2f} confidence)"



In [25]:
text = input()
print(predict_spam(str(text)))

We have published a new free template today.     	 Free Interlinked Chain Diagram PowerPoint Template Please Enable Images for this Email - Here we will show you a thumbnail image for the free template  View & Download     	      	  	 Upgrade to Premium Access Latest PowerPoint templates  With a premium subscription you can choose from over 50,000 editable PowerPoint templates and finish your presentations in less time. Click here to apply a special coupon code and become premium today.  Upgrade Now
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
The email is a spam (0.93 confidence)
